In [ ]:
import json
import glob
import os
import pandas as pd
import numpy as np
import xlsxwriter
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from sklearn.model_selection import train_test_split  
from sklearn.ensemble import RandomForestRegressor  
from sklearn.metrics import mean_squared_error  
import seaborn as sns

## Overall residue alignment

In [ ]:
def cumulative_aligned_ratio(df):
    sorted_df = df.sort_values(by="aligned_fraction", ascending=False)
    sorted_df["numb"] = 1
    total_size = sorted_df.shape[0]
    sorted_df["cum_frac"] = sorted_df["numb"].cumsum() / total_size
    return sorted_df[["aligned_fraction", "cum_frac"]]

In [ ]:
tool_name = {"fs3di": "Foldseek (3Di)", "fs": "Foldseek", "mm": "MMseqs", "rs": "Reseek", "tm": "TM-align", "hmmscan": "Hmmscan"}
fig_dir = "../figures/"

In [ ]:
plt.figure(dpi=300)
paths = sorted(glob.glob("../data/processed/residue_ali_frac_per_seed_split_vs_split/*_all.tsv"))

for path in paths:
    df = pd.read_csv(path, sep="\t")
    cum_df = cumulative_aligned_ratio(df)
    tool_abbreviation = os.path.basename(path).split("_")[0]
    y_axis = np.insert(cum_df["aligned_fraction"], 0, 1) # 1 is inserted in the beginning to make sure AUC calculation considers y=1 for small x values
    x_axis = np.insert(cum_df["cum_frac"], 0, 0)                   # 1 is inserted in the beginning to make sure AUC calculation considers y=1 for small x values
    auc = np.trapz( y_axis, x_axis) 
    plt.plot(x_axis, y_axis, label=f"{tool_name[tool_abbreviation]} (AUC = {auc:.2f})")
plt.xlabel("Fraction of seeds")
plt.ylabel("Fraction of correctly aligned residues")
plt.legend()
plt.savefig(f"{fig_dir}/overall_residue_alignment_split_vs_split.png")
plt.show()

## Conserved residues alignment

In [ ]:
plt.figure(dpi=300)
paths = sorted(glob.glob("../data/processed/residue_ali_frac_per_seed_split_vs_split/*_conserved.tsv"))

for path in paths:
    df = pd.read_csv(path, sep="\t")
    cum_df = cumulative_aligned_ratio(df)
    tool_abbreviation = os.path.basename(path).split("_")[0]
    y_axis = np.insert(cum_df["aligned_fraction"], 0, 1) # 1 is inserted in the beginning to make sure AUC calculation considers y=1 for small x values
    x_axis = np.insert(cum_df["cum_frac"], 0, 0)                   # 1 is inserted in the beginning to make sure AUC calculation considers y=1 for small x values
    auc = np.trapz( y_axis, x_axis) 
    plt.plot(x_axis, y_axis, label=f"{tool_name[tool_abbreviation]} (AUC = {auc:.2f})")
plt.xlabel("Fraction of seeds")
plt.ylabel("Fraction of correctly aligned residues")
plt.legend()
plt.savefig(f"{fig_dir}/conserved_residue_alignment_split_vs_split.png")
plt.show()

## Active site alignments

In [ ]:
plt.figure(dpi=300)
paths = sorted(glob.glob("../data/processed/residue_ali_frac_per_seed_split_vs_split/*_active_site.tsv"))

for path in paths:
    df = pd.read_csv(path, sep="\t")
    cum_df = cumulative_aligned_ratio(df)
    tool_abbreviation = os.path.basename(path).split("_")[0]
    y_axis = np.insert(cum_df["aligned_fraction"], 0, 1) # 1 is inserted in the beginning to make sure AUC calculation considers y=1 for small x values
    x_axis = np.insert(cum_df["cum_frac"], 0, 0)                   # 1 is inserted in the beginning to make sure AUC calculation considers y=1 for small x values
    auc = np.trapz( y_axis, x_axis) 
    plt.plot(x_axis, y_axis, label=f"{tool_name[tool_abbreviation]} (AUC = {auc:.2f})")
plt.xlabel("Fraction of seeds")
plt.ylabel("Fraction of correctly aligned residues")
plt.legend()
plt.savefig(f"{fig_dir}/active_site_alignment_split_vs_split.png")
plt.show()

## Binding site alignment

In [ ]:
plt.figure(dpi=300)
paths = sorted(glob.glob("../data/processed/residue_ali_frac_per_seed_split_vs_split/*_binding_site.tsv"))

for path in paths:
    df = pd.read_csv(path, sep="\t")
    cum_df = cumulative_aligned_ratio(df)
    tool_abbreviation = os.path.basename(path).split("_")[0]
    y_axis = np.insert(cum_df["aligned_fraction"], 0, 1) # 1 is inserted in the beginning to make sure AUC calculation considers y=1 for small x values
    x_axis = np.insert(cum_df["cum_frac"], 0, 0)                   # 1 is inserted in the beginning to make sure AUC calculation considers y=1 for small x values
    auc = np.trapz( y_axis, x_axis) 
    plt.plot(x_axis, y_axis, label=f"{tool_name[tool_abbreviation]} (AUC = {auc:.2f})")
plt.xlabel("Fraction of seeds")
plt.ylabel("Fraction of correctly aligned residues")
plt.legend()

plt.savefig(f"{fig_dir}/binding_site_alignment_split_vs_split.png")
plt.show()

Direct comparison of previous plots shows that there is a higher chance to correct the conserved residues with each other correctly, compared to the background. One hypothesis is that the structures around the conserved sites are modeled with a high accuracy. This can be judged based on pLDDT. In the next step, we are going to find the difference between the pLDDT of conserved residues and those of the background.

In [ ]:
all_plddts = json.loads(open("../data/pfam_plddt.json").read())
conserved_residues = json.loads(open("../tmp/residue_features/conserved.json").read())
def avg(num_list):
    return sum(num_list)/len(num_list)

cons_minus_bg_plddt = {}
for seed_id, con_res in conserved_residues.items():
    if len(con_res) == 0:
        continue
    bkg_plddt = all_plddts[seed_id]
    bkg_avg_plddt = avg(bkg_plddt)
    conserved_residues_avg_plddt = avg([bkg_plddt[i-1] for i in con_res])
    cons_minus_bg_plddt[seed_id] = conserved_residues_avg_plddt - bkg_avg_plddt

data = pd.DataFrame({ "seed_id": cons_minus_bg_plddt.keys(), "conserved_residue_plddt_minus_bkg": cons_minus_bg_plddt.values()})
sorted_data = data.sort_values(by="conserved_residue_plddt_minus_bkg" ,ascending = False).reset_index(drop=True)

more_confident_than_background = (sorted_data["conserved_residue_plddt_minus_bkg"] > 0).argmin() / len(sorted_data)
print(f"For {round(100 * more_confident_than_background, 2)} percent of the seeds, the pLDDT of the conserved sites is higher than the rest of the seed")

seeds_with_low_confidence_conserved_sites = sorted_data[sorted_data["conserved_residue_plddt_minus_bkg"]<0]["seed_id"]

## Residue alignment for low confidence conserved residues

In [ ]:
plt.figure(dpi=300)
paths = sorted(glob.glob("../data/processed/residue_ali_frac_per_seed_split_vs_split/*_conserved.tsv"))

for path in paths:
    df = pd.read_csv(path, sep="\t")
    df = df[df["query"].isin(seeds_with_low_confidence_conserved_sites)]    # This is for selecting the seeds whose pLDDT of conserved residues is smaller than the background
    cum_df = cumulative_aligned_ratio(df)
    tool_abbreviation = os.path.basename(path).split("_")[0]
    y_axis = np.insert(cum_df["aligned_fraction"], 0, 1) # 1 is inserted in the beginning to make sure AUC calculation considers y=1 for small x values
    x_axis = np.insert(cum_df["cum_frac"], 0, 0)                   # 1 is inserted in the beginning to make sure AUC calculation considers y=1 for small x values
    auc = np.trapz( y_axis, x_axis) 
    plt.plot(x_axis, y_axis, label=f"{tool_name[tool_abbreviation]} (AUC = {auc:.2f})")
plt.xlabel("Fraction of seeds")
plt.ylabel("Fraction of correctly aligned residues")
plt.legend()
plt.savefig(f"{fig_dir}/low_plddt_conserved_sites_alignment_split_vs_split.png")
plt.show()



In [ ]:
def train_model(data_df):
    """Trains a random forest for each model and shows the importance of the features"""
    # Assuming the last 8 columns are features and the second column is the query  
    X = data_df.iloc[:, 1:-1]     # Features (starting from second column till one to the last columns)  
    y = data_df.iloc[:, -1]       # query (second column)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Initialize the model  
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)  
    
    # Fit the model  
    rf_model.fit(X_train, y_train)
    # Make predictions  
    y_pred_train = rf_model.predict(X_train)
    y_pred_test = rf_model.predict(X_test)  
    
    # Calculate Mean Squared Error  
    mse_train = mean_squared_error(y_train, y_pred_train)
    mse_test = mean_squared_error(y_test, y_pred_test)
    
    importances = rf_model.feature_importances_  
    # Create a DataFrame for visualization  
    importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})  
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    
    return {"model":rf_model, "mse_train": mse_train, "mse_test": mse_test, "feature_importance": importance_df}

In [ ]:
data_dir = "../data/"
pi_df = pd.read_csv(f"{data_dir}/processed/avg_intra_fam_pident.tsv", sep="\t") #pi means percentage identity
ss_info_df = pd.read_csv(f"{data_dir}/processed/ss_info_pfam.tsv", sep="\t")
cn_df = pd.read_csv(f"{data_dir}/processed/avg_contact_num.tsv", sep="\t")
plddt_df = pd.read_csv(f"{data_dir}/processed/pfam_avg_plddt.tsv", sep="\t")
plddt_df["size"] = plddt_df["seed_id"].str.split("-", expand=True)[2].astype(int) - plddt_df["seed_id"].str.split("-", expand=True)[1].astype(int) + 1
protperties_df = pi_df.merge(ss_info_df, on="seed_id").merge(cn_df, on="seed_id").merge(plddt_df, on="seed_id")

## Train a model for all residues

In [ ]:
paths = sorted(glob.glob("../data/processed/residue_ali_frac_per_seed_split_vs_split/*_all.tsv"))
model_info = {}
for path in paths:
    tool = os.path.basename(path).replace("_all.tsv", "")
    perf_df = pd.read_csv(path, sep="\t").rename(columns={"query": "seed_id"})
    properties_perf = protperties_df.merge(perf_df, on = "seed_id")
    data_type = os.path.basename(path).replace(".tsv", "")
    model_info[data_type] = train_model(properties_perf)
    print(f"just processed {data_type}")

In [ ]:
for model in model_info.keys():
    print(model)
    print(model_info[model])

## Train a model for conserved residues alignment

In [ ]:
paths = sorted(glob.glob("../data/processed/residue_ali_frac_per_seed_split_vs_split/*_conserved.tsv"))
for path in paths:
    tool = os.path.basename(path).replace("_conserved.tsv", "")
    perf_df = pd.read_csv(path, sep="\t").rename(columns={"query": "seed_id"})
    properties_perf = protperties_df.merge(perf_df, on = "seed_id")
    data_type = os.path.basename(path).replace(".tsv", "")
    model_info[data_type] = train_model(properties_perf)
    print(f"just processed {data_type}")

In [ ]:
for model in model_info.keys():
    if model.endswith("_conserved"):
        print(model)
        print(model_info[model])